In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
pd.set_option('display.max_columns', 99)

In [3]:
superstore = pd.read_csv('./data/superstore_dataset2011-2015.csv.zip', encoding = "ISO-8859-1")

In [4]:
superstore.shape

(51290, 24)

In [5]:
superstore.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium
1,22253,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
2,48883,HU-2011-1220,1/1/2011,5/1/2011,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High
3,11731,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
4,22255,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium


In [6]:
superstore.columns = superstore.columns.str.replace(' ', '_').str.replace('-', '_').str.lower()

In [7]:
superstore.isna().sum()

row_id                0
order_id              0
order_date            0
ship_date             0
ship_mode             0
customer_id           0
customer_name         0
segment               0
city                  0
state                 0
country               0
postal_code       41296
market                0
region                0
product_id            0
category              0
sub_category          0
product_name          0
sales                 0
quantity              0
discount              0
profit                0
shipping_cost         0
order_priority        0
dtype: int64

In [8]:
superstore.dtypes

row_id              int64
order_id           object
order_date         object
ship_date          object
ship_mode          object
customer_id        object
customer_name      object
segment            object
city               object
state              object
country            object
postal_code       float64
market             object
region             object
product_id         object
category           object
sub_category       object
product_name       object
sales             float64
quantity            int64
discount          float64
profit            float64
shipping_cost     float64
order_priority     object
dtype: object

In [9]:
superstore.drop('postal_code', axis=1, inplace=True)

In [10]:
db_uri = "sqlite:///data/superstore.sqlite"
engine = sa.create_engine(db_uri)

In [11]:
# create table
engine.execute('''
    CREATE TABLE "superstore" (
    row_id INTEGER NOT NULL,
    order_id TEXT NOT NULL,
    order_date TEXT,
    ship_date TEXT,
    ship_mode TEXT,
    customer_id TEXT,
    customer_name TEXT,
    segment TEXT,
    city TEXT,
    state TEXT,
    country TEXT,
    market TEXT,
    region TEXT,
    product_id TEXT,
    category TEXT,
    sub_category TEXT,
    product_name TEXT,
    sales REAL,
    quantity INTEGER,
    discount REAL,
    profit REAL,
    shipping_cost REAL,
    order_priority TEXT
    )
''')
    

In [12]:
engine.execute('''
    INSERT INTO "superstore"
    (row_id, order_id, order_date, ship_date, ship_mode, customer_id, customer_name, segment, city,
     state, country, market, region, product_id, category, sub_category, product_name,
     sales, quantity, discount, profit, shipping_cost, order_priority)
    VALUES ({row_id}, "{order_id}", "{order_date}", "{ship_date}", "{ship_mode}", 
    "{customer_id}", "{customer_name}", "{segment}", "{city}",
     "{state}", "{country}", "{market}", "{region}", "{product_id}", 
     "{category}", "{sub_category}", "{product_name}",
     {sales}, {quantity}, {discount}, {profit}, {shipping_cost}, "{order_priority}")

'''.format(**superstore.iloc[0].to_dict()))

In [13]:
superstore_0 = pd.read_sql_table('superstore', engine)

In [14]:
superstore_0

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,market,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,42433,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.3,2,0.0,106.14,35.46,Medium


In [19]:
engine.execute('drop table "superstore"')

In [20]:
# create table
engine.execute('''
    CREATE TABLE "superstore" (
    row_id INTEGER NOT NULL,
    order_id TEXT NOT NULL,
    order_date TEXT,
    ship_date TEXT,
    ship_mode TEXT,
    customer_id TEXT,
    customer_name TEXT,
    segment TEXT,
    city TEXT,
    state TEXT,
    country TEXT,
    market TEXT,
    region TEXT,
    product_id TEXT,
    category TEXT,
    sub_category TEXT,
    product_name TEXT,
    sales REAL,
    quantity INTEGER,
    discount REAL,
    profit REAL,
    shipping_cost REAL,
    order_priority TEXT
    )
''')
    

In [21]:
table = sa.Table("superstore", sa.MetaData(engine), autoload=True, autoload_with=engine)
for _, row in superstore.iterrows():
    with engine.begin() as connection:
        row_dict = row.to_dict()
        connection.execute(table.insert().values(row_dict))